# Ative o uso da GPU em:
Runtime > Change runtime type > Hardware Accelerator > GPU > Save

In [1]:
!nvidia-smi

Tue Nov  1 14:25:13 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 520.61.05    Driver Version: 520.61.05    CUDA Version: 11.8     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  On   | 00000000:01:00.0 Off |                  N/A |
| N/A   52C    P3    N/A /  N/A |      6MiB /  4096MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0


In [3]:
!pip3 install git+https://github.com/andreinechaev/nvcc4jupyter.git

  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-yvdtzxv1
  Running command git clone -q https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-yvdtzxv1
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit aac710a35f52bb78ab34d2e52517237941399eff


In [4]:
%load_ext nvcc_plugin

directory /home/m4rquee/projects/secomp22/notebook/src already exists
Out bin /home/m4rquee/projects/secomp22/notebook/result.out


# "Hello World" em CUDA:

In [5]:
%%cu
#include <stdio.h>

//#   Atributo  Executa no  Chamado por  Nota
//# __global__  Device      Host         Precisa ser void
//# __device__  Device      Device       Pode retornar qualquer tipo
//# __host__    Host        Host         Opcional

__global__ void hello() {
    printf("Hello World da GPU!\n");
}

int main() {
    hello<<<1, 1>>>(); // <<<blocos, threads por bloco>>>, executa de forma assíncrona!
    printf("Hello World da CPU!\n");
    cudaDeviceSynchronize(); // espera até o kernel finalizar
    return 0;
}

Hello World da CPU!
Hello World da GPU!



# Primeiro exemplo - soma de vetores:

In [6]:
%%cu
#include <math.h>
#include <stdio.h>
#include <stdlib.h>
#include <assert.h>

#define N 100000000
#define ERRO_MAX 1e-6

__global__ void soma_vetores(float *out, float *a, float *b, int n) {
    for (int i = 0; i < n; i++)
        out[i] = a[i] + b[i];
}

int main() {
    float *a, *b, *out; // vetores na memória RAM 
    float *d_a, *d_b, *d_out; // vetores na memória da GPU

    // Aloca os vetores no host (memória RAM):
    a   = (float*) malloc(sizeof(float) * N);
    b   = (float*) malloc(sizeof(float) * N);
    out = (float*) malloc(sizeof(float) * N);

    // Inicializa os vetores:
    for (int i = 0; i < N; i++) {
        a[i] = 1.0f;
        b[i] = 2.0f;
    }

    // Aloca os vetores no device (memória da GPU):
    cudaMalloc((void**) &d_a, sizeof(float) * N);
    cudaMalloc((void**) &d_b, sizeof(float) * N);
    cudaMalloc((void**) &d_out, sizeof(float) * N);

    //###### TRANSFERÊNCIA DE DADOS ######
    // Transfere os dados para a GPU:
    cudaMemcpy(d_a, a, sizeof(float) * N, cudaMemcpyHostToDevice);
    cudaMemcpy(d_b, b, sizeof(float) * N, cudaMemcpyHostToDevice);

    //############ COMPUTAÇÃO ############
    soma_vetores<<<1, 1>>>(d_out, d_a, d_b, N); // executa a soma
    cudaDeviceSynchronize(); // espera até o kernel finalizar
    
    //###### TRANSFERÊNCIA DE DADOS ######
    // Transfere os dados de volta da GPU:
    cudaMemcpy(out, d_out, sizeof(float) * N, cudaMemcpyDeviceToHost);

    // Testa a resposta:
    for (int i = 0; i < N; i++)
        assert(fabs(out[i] - a[i] - b[i]) < ERRO_MAX);
    printf("out[0] = %.3f\n", out[0]);
    printf("A soma funcionou!\n");

    // Libera os vetores:
    cudaFree(d_a);
    cudaFree(d_b);
    cudaFree(d_out);
    free(a); 
    free(b);
    free(out);

    return 0;
}

out[0] = 3.000
A soma funcionou!



# Segundo exemplo - soma de vetores paralela:

In [7]:
%%cu
#include <math.h>
#include <stdio.h>
#include <stdlib.h>
#include <assert.h>

#define N 100000000
#define ERRO_MAX 1e-6

__global__ void soma_vetores(float *out, float *a, float *b, int n) {
    int i = blockDim.x * blockIdx.x + threadIdx.x;
    if (i < n)
        out[i] = a[i] + b[i];
}

int main() {
    float *a, *b, *out; // vetores na memória RAM 
    float *d_a, *d_b, *d_out; // vetores na memória da GPU

    // Aloca os vetores no host (memória RAM):
    a   = (float*) malloc(sizeof(float) * N);
    b   = (float*) malloc(sizeof(float) * N);
    out = (float*) malloc(sizeof(float) * N);

    // Inicializa os vetores:
    for (int i = 0; i < N; i++) {
        a[i] = 1.0f;
        b[i] = 2.0f;
    }

    // Aloca os vetores no device (memória da GPU):
    cudaMalloc((void**) &d_a, sizeof(float) * N);
    cudaMalloc((void**) &d_b, sizeof(float) * N);
    cudaMalloc((void**) &d_out, sizeof(float) * N);

    //###### TRANSFERÊNCIA DE DADOS ######
    // Transfere os dados para a GPU:
    cudaMemcpy(d_a, a, sizeof(float) * N, cudaMemcpyHostToDevice);
    cudaMemcpy(d_b, b, sizeof(float) * N, cudaMemcpyHostToDevice);

    //############ COMPUTAÇÃO ############
    int threads = 1024;
    int blocos = ((N + threads) / threads);
    soma_vetores<<<blocos, threads>>>(d_out, d_a, d_b, N); // executa a soma
    cudaDeviceSynchronize(); // espera até o kernel finalizar
    
    //###### TRANSFERÊNCIA DE DADOS ######
    // Transfere os dados de volta da GPU:
    cudaMemcpy(out, d_out, sizeof(float) * N, cudaMemcpyDeviceToHost);

    // Testa a resposta:
    for(int i = 0; i < N; i++)
        assert(fabs(out[i] - a[i] - b[i]) < ERRO_MAX);
    printf("out[0] = %.3f\n", out[0]);
    printf("A soma funcionou!\n");

    // Libera os vetores:
    cudaFree(d_a);
    cudaFree(d_b);
    cudaFree(d_out);
    free(a); 
    free(b);
    free(out);

    return 0;
}

out[0] = 3.000
A soma funcionou!

